In [1]:
from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt

import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline

**Login

In [2]:
username = "jeroendereu"
password = "***"

In [3]:
api = SentinelAPI(username, password, "https://scihub.copernicus.eu/dhus")

**Basics

In [4]:
product_id_example = "36871bc0-b115-4dd8-9830-392619fc9e25"

In [ ]:
# download single scene by known product id
api.download(product_id_example)

Downloading:  38%|████████▊              | 2.42G/6.33G [53:15<2:07:49, 510KB/s]

In [ ]:
# Get basic information about the product: its title, file size, MD5 sum, date, footprint and its download url
# api.get_product_odata(product_id_example)

In [ ]:
# Get the product's full metadata available on the server
# api.get_product_odata(product_id_example, full=True)

In [ ]:
# define footprint = search the data by polygon
footprint_flanders_geojson = geojson_to_wkt(read_geojson("C:/WERKMAP/Anaconda/sentinelsat/footprint_flanders_geojson.geojson"))

In [ ]:
footprint_flanders_geojson

In [ ]:
plot_geojson = gpd.read_file(footprint_flanders_geojson)
plot_geojson.plot()

In [ ]:
footprint_flanders_coordinates = 'POLYGON ((2.5406630 50.6879597, 2.5406630 51.5056400, 5.9098982 51.5056400, 5.9098982 50.6879597, 2.5406630 50.6879597))'
# coordinates maximum extent Flanders

In [ ]:
footprint_flanders_coordinates

In [ ]:
footprint_flanders_geojson == footprint_flanders_coordinates

In [ ]:
# parameters
startdate = '20160301'
enddate = '20161031'
platform = 'Sentinel-2'
min_cloud = '0'
max_cloud = '30'

In [ ]:
# query the data by footprint, time, and keywords
products = api.query(footprint_flanders_geojson, 
                     startdate, 
                     enddate, 
                     platformname = platform, 
                     cloudcoverpercentage = '[' + min_cloud + ' TO ' + max_cloud + ']'
                    )

In [ ]:
# queried data is stored in dictionary
products

In [ ]:
# download all datasets (from query)
# api.download_all(products)

In [ ]:
# GeoJSON FeatureCollection containing footprints and metadata of the scenes
geojson_products = api.to_geojson(products)

In [ ]:
geojson_products

In [ ]:
# GeoPandas GeoDataFrame with the metadata of the scenes and the footprints as geometries
api.to_geodataframe(products)

**jp2 naar geotiff

In [ ]:
import os
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline
import rasterio
import rsgislib
from rsgislib import imageutils

In [ ]:
input_directory = "C:/WERKMAP/Anaconda/sentinelsat/downloaded_datasets/S2A_OPER_PRD_MSIL1C_PDMC_20161011T191935_R094_V20161011T105952_20161011T110223.SAFE/GRANULE/S2A_OPER_MSI_L1C_TL_SGS__20161011T162433_A006812_T31UES_N02.04/IMG_DATA/"

In [ ]:
for path in glob(os.path.join(input_directory, "**", "*.jp2"), recursive=True):
    print(path)

In [ ]:
images = []

In [ ]:
for path in glob(os.path.join(input_directory, "**", "*.jp2"),recursive=True):
    images.append(path)

In [ ]:
images

In [ ]:
band_names = []

In [ ]:
for path in glob(os.path.join(input_directory, "**", "*.jp2"),recursive=True):
    basename = os.path.basename(path)
    filename, file_extention = os.path.splitext(basename)
    band_names.append(filename)

In [ ]:
band_names

In [ ]:
out_image = "C:/WERKMAP/Anaconda/sentinelsat/sentinel_image_stack.tif"
data_type = rsgislib.TYPE_16UINT
imageutils.stackImageBands(images, band_names, out_image, None, 0, 'Gtiff', data_type)

**Clippen

In [ ]:
inputImage = out_image
inputVector = "C:/WERKMAP/Anaconda/shapefile_geojson_shapefile/example_shapefiles/polygon_example.shp"
outputImage = "C:/WERKMAP/Anaconda/sentinelsat/sentinel_image_stack_clipped.tif"
gdalformat = 'Gtiff'
datatype = rsgislib.TYPE_32FLOAT
imageutils.subset(inputImage, inputVector, outputImage, gdalformat, datatype)

other possibility: via bounding box (Xmin, Xmax, Ymin, Ymax)

import geojson as gj
from osgeo import ogr, osr, gdal

# Enable GDAL/OGR exceptions
gdal.UseExceptions()

# GDAL & OGR memory drivers
GDAL_MEMORY_DRIVER = gdal.GetDriverByName('MEM')
OGR_MEMORY_DRIVER = ogr.GetDriverByName('Memory')

def cut_by_geojson(input_file, output_file, shape_geojson):

    # Get coords for bounding box
    x, y = zip(*gj.utils.coords(gj.loads(shape_geojson)))
    min_x, max_x, min_y, max_y = min(x), max(x), min(y), max(y)

    # Open original data as read only
    dataset = gdal.Open(input_file, gdal.GA_ReadOnly)

    bands = dataset.RasterCount

    # Getting georeference info
    transform = dataset.GetGeoTransform()
    projection = dataset.GetProjection()
    xOrigin = transform[0]
    yOrigin = transform[3]
    pixelWidth = transform[1]
    pixelHeight = -transform[5]

    # Getting spatial reference of input raster
    srs = osr.SpatialReference()
    srs.ImportFromWkt(projection)

    # WGS84 projection reference
    OSR_WGS84_REF = osr.SpatialReference()
    OSR_WGS84_REF.ImportFromEPSG(4326)

    # OSR transformation
    wgs84_to_image_trasformation = osr.CoordinateTransformation(OSR_WGS84_REF,
                                                                srs)
    XYmin = wgs84_to_image_trasformation.TransformPoint(min_x, max_y)
    XYmax = wgs84_to_image_trasformation.TransformPoint(max_x, min_y)

    # Computing Point1(i1,j1), Point2(i2,j2)
    i1 = int((XYmin[0] - xOrigin) / pixelWidth)
    j1 = int((yOrigin - XYmin[1]) / pixelHeight)
    i2 = int((XYmax[0] - xOrigin) / pixelWidth)
    j2 = int((yOrigin - XYmax[1]) / pixelHeight)
    new_cols = i2 - i1 + 1
    new_rows = j2 - j1 + 1

    # New upper-left X,Y values
    new_x = xOrigin + i1 * pixelWidth
    new_y = yOrigin - j1 * pixelHeight
    new_transform = (new_x, transform[1], transform[2], new_y, transform[4],
                     transform[5])

    wkt_geom = ogr.CreateGeometryFromJson(str(field_shape_geojson))
    wkt_geom.Transform(wgs84_to_image_trasformation)

    target_ds = GDAL_MEMORY_DRIVER.Create('', new_cols, new_rows, 1,
                                          gdal.GDT_Byte)
    target_ds.SetGeoTransform(new_transform)
    target_ds.SetProjection(projection)

    # Create a memory layer to rasterize from.
    ogr_dataset = OGR_MEMORY_DRIVER.CreateDataSource('shapemask')
    ogr_layer = ogr_dataset.CreateLayer('shapemask', srs=srs)
    ogr_feature = ogr.Feature(ogr_layer.GetLayerDefn())
    ogr_feature.SetGeometryDirectly(ogr.Geometry(wkt=wkt_geom.ExportToWkt()))
    ogr_layer.CreateFeature(ogr_feature)

    gdal.RasterizeLayer(target_ds, [1], ogr_layer, burn_values=[1],
                        options=["ALL_TOUCHED=TRUE"])

    # Create output file
    driver = gdal.GetDriverByName('GTiff')
    outds = driver.Create(output_file, new_cols, new_rows, bands,
                          gdal.GDT_Float32)

    # Read in bands and store all the data in bandList
    mask_array = target_ds.GetRasterBand(1).ReadAsArray()
    band_list = []

    for i in range(bands):
        band_list.append(dataset.GetRasterBand(i + 1).ReadAsArray(i1, j1,
                         new_cols, new_rows))

    for j in range(bands):
        data = np.where(mask_array == 1, band_list[j], mask_array)
        outds.GetRasterBand(j + 1).SetNoDataValue(0)
        outds.GetRasterBand(j + 1).WriteArray(data)

    outds.SetProjection(projection)
    outds.SetGeoTransform(new_transform)

    target_ds = None
    dataset = None
    outds = None
    ogr_dataset = None

input_file = "C:/WERKMAP/Anaconda/sentinelsat/sentinel_image_stack.tif"
output_file = "C:/WERKMAP/Anaconda/sentinelsat/sentinel_image_stack_clipped.tif"
shape_geojson = footprint_flanders_coordinates

cut_by_geojson(input_file, output_file, shape_geojson)